In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import precision_recall_curve, recall_score

In [77]:
churn_data = pd.read_csv('/Users/leahdeyoung/Desktop/GitHub/beta-bank-practicum/Churn.csv', encoding = "utf-8")


In [78]:
display(churn_data.sample(10))
churn_data.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
7209,7210,15788131,Atkins,653,France,Male,47,NaN,0.00,1,1,0,50695.93,1
2129,2130,15664881,Norton,702,France,Male,34,4.0,100054.77,1,1,0,109496.45,0
6794,6795,15694098,Jackson,575,France,Female,54,NaN,68332.96,1,1,1,144390.75,0
4862,4863,15686780,Rogova,645,Spain,Female,55,1.0,133676.65,1,0,1,17095.49,0
6717,6718,15586674,Shaw,663,Spain,Female,58,5.0,216109.88,1,0,1,74176.71,1
3772,3773,15699486,Johnson,745,Spain,Male,34,7.0,132944.53,1,1,1,31802.92,0
7499,7500,15790113,Schofield,609,Germany,Female,71,6.0,113317.10,1,1,0,108258.22,1
4970,4971,15614893,Meng,689,Spain,Male,38,2.0,0.00,1,1,1,82709.80,0
981,982,15566156,Franklin,749,Germany,Female,44,NaN,71497.79,2,0,0,151083.80,0
4835,4836,15590623,Kovalyov,561,Spain,Male,34,4.0,85141.79,2,1,1,29217.37,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [79]:
print(churn_data.duplicated().sum())

0


In [80]:
churn_data.columns = churn_data.columns.str.lower()
churn_data = churn_data.rename(columns={
    'rownumber': 'row_number', 
    'customerid': 'customer_id',
    'creditscore': 'credit_score',
    'numofproducts': 'num_of_products',
    'hascrcard': 'has_cr_card',
    'isactivemember': 'is_active_member',
    'estimatedsalary': 'estimated_salary',
})
print(churn_data.columns)

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')


In [81]:
print(churn_data['customer_id'].duplicated().sum())
churn_data['surname_lowercase'] = churn_data['surname'].str.lower()

print(churn_data['surname_lowercase'].duplicated().sum())
print(churn_data.duplicated(subset=['surname_lowercase', 'gender', 'age']).sum())
print(churn_data.duplicated(subset=['surname_lowercase', 'gender', 'age', 'geography']).sum())
print(churn_data.duplicated(subset=['surname_lowercase', 'gender', 'age', 'geography', 'credit_score']).sum())
print(churn_data.duplicated(subset=['surname_lowercase', 'gender', 'age', 'geography', 'credit_score', 'estimated_salary']).sum())






0
7069
596
241
2
0


In [82]:
print(churn_data['tenure'].isna().value_counts())
print(churn_data.query("tenure.isna() & is_active_member == 0").count())
churn_data['tenure'] = churn_data['tenure'].where((churn_data['tenure'].notna() & churn_data['is_active_member'] == 1) , 0.0) 
print()
print(churn_data.query("tenure.isna() & is_active_member == 0").count())
print(churn_data['tenure'].isna().value_counts())
churn_data['tenure'] = pd.to_numeric(churn_data['tenure'], errors='coerce')



False    9091
True      909
Name: tenure, dtype: int64
row_number           445
customer_id          445
surname              445
credit_score         445
geography            445
gender               445
age                  445
tenure                 0
balance              445
num_of_products      445
has_cr_card          445
is_active_member     445
estimated_salary     445
exited               445
surname_lowercase    445
dtype: int64

row_number           0
customer_id          0
surname              0
credit_score         0
geography            0
gender               0
age                  0
tenure               0
balance              0
num_of_products      0
has_cr_card          0
is_active_member     0
estimated_salary     0
exited               0
surname_lowercase    0
dtype: int64
False    10000
Name: tenure, dtype: int64


In [83]:
churn_data['exited'] = churn_data['exited'].astype('boolean')
